In [ ]:
import pymc as pm
import pandas as pd
import arviz as az

import matplotlib.pyplot as plt
import seaborn as sns

import warnings
warnings.filterwarnings("ignore", category=FutureWarning)

In [ ]:
def plot_kde(sample, **options):
    """Plot a distribution using KDE.
    
    sample: sequence of values
    """
    sns.kdeplot(sample, cut=0, **options)
    
def legend(**options):
    """Make a legend only if there are labels."""
    handles, labels = plt.gca().get_legend_handles_labels()
    if len(labels):
        plt.legend(**options)
    
def decorate_proportion(ylabel='Likelihood'):
    """Decorate the axes."""
    plt.xlabel('Proportion of deaths (x)')
    plt.ylabel(ylabel)
    plt.title('Distribution of proportion')
    legend()
    
def plot_hist(sample, **options):
    """Plot a histogram of goals.
    
    sample: sequence of values
    """
    sns.histplot(sample, stat='probability', discrete=True,
                 alpha=0.5, **options)
    
def decorate_heads(ylabel='Probability'):
    """Decorate the axes."""
    plt.xlabel('Number of deaths (k)')
    plt.ylabel(ylabel)
    plt.title('Distribution of deaths')
    legend()

In [ ]:
df = pd.read_csv('Data/apache_sofa_treatment_63.csv', index_col=0)
df.columns

In [ ]:
df_1y = df[df['1-year']==1]
df_3y = df[df['3-year']==1]

In [ ]:
n = df.shape[0]
with pm.Model() as model1:
    x1 = pm.Beta('x1', alpha=2, beta=2)
    x2 = pm.Beta('x2', alpha=2, beta=2)
    k1 = pm.Binomial('k1', n=n, p=x1, observed=df_1y.shape[0])
    k2 = pm.Binomial('k2', n=n, p=x2, observed=df_3y.shape[0])   

In [ ]:
pm.model_to_graphviz(model1)

In [ ]:
with model1:
    idata1 = pm.sample(1000)

In [ ]:
with model1:
    az.plot_posterior(idata1)

In [ ]:
with pm.Model() as model2:
    alpha = pm.Gamma('alpha', alpha=4, beta=0.5)
    beta = pm.Gamma('beta', alpha=4, beta=0.5)
    x1 = pm.Beta('x1', alpha, beta)
    x2 = pm.Beta('x2', alpha, beta)
    k1 = pm.Binomial('k1', n=n, p=x1, observed=df_1y.shape[0])
    k2 = pm.Binomial('k2', n=n, p=x2, observed=df_3y.shape[0])   

In [ ]:
pm.model_to_graphviz(model2)

In [ ]:
with model2:
    idata2 = pm.sample_prior_predictive(500)

In [ ]:
idata2

In [ ]:
plot_kde(idata2.prior['alpha'][0], label='alpha')
plot_kde(idata2.prior['beta'][0], label='beta')
plt.xlabel('Hyperparameter')
plt.ylabel('Likelihood')
plt.title('Distribution of hyperparameters')
plt.legend();

In [ ]:
sample_prior_x1 = idata2.prior['x1']
sample_prior_x2 = idata2.prior['x2']
sample_prior_x1.values.mean(), sample_prior_x2.values.mean()

In [ ]:
plot_kde(sample_prior_x1.values[0], color='orange', label='prior x1')
plot_kde(sample_prior_x2.values[0], color='blue', label='prior x2')
decorate_proportion()

In [ ]:
sample_prior_pred_k1 = idata2.prior_predictive['k1']
sample_prior_pred_k2 = idata2.prior_predictive['k2']
sample_prior_pred_k1.values.mean(), sample_prior_pred_k2.values.mean()

In [ ]:
plot_hist(sample_prior_pred_k1[0], label='k1')
decorate_heads()

In [ ]:
with model2:
    idata2 = pm.sample(1000)

In [ ]:
with model2:
    az.plot_trace(idata2)

In [ ]:
with model2:
    az.plot_posterior(idata2, var_names=['x1', 'x2'])

In [ ]:
with model2:
    az.plot_posterior(idata2, var_names=['alpha', 'beta'])

In [ ]:
idata2.posterior['alpha'][0].values

In [ ]:
alphas = idata2.posterior['alpha'][0].values
betas = idata2.posterior['beta'][0].values

sample_posterior_x = pm.Beta.dist(alphas, betas)
sample_posterior_x = pm.draw(sample_posterior_x, 1000)
sample_posterior_x.mean()

In [ ]:
plot_kde(sample_prior_x1.values[0], color='gray', label='prior')
plot_kde(sample_posterior_x[0], label='posterior')
decorate_proportion()

In [ ]:
df

In [ ]:
df = pd.read_csv('Data/apache_sofa_all.csv')

In [ ]:
import numpy as np

In [ ]:
df = df.groupby('Rozpoznania')[['30-day']].agg(['count', 'sum'])

In [ ]:
df.columns = df.columns.droplevel(0)
df = df.reset_index()
df = df.rename(columns={'Rozpoznania': 'Rozp',
                        'count': 'Cases', 
                        'sum': 'Deaths'})
df['Death %'] = df.apply(lambda x: np.round(x['Deaths']/x['Cases'], 4)*100, axis=1)

In [ ]:
df

In [ ]:
data_ns = df['Cases'].values
data_ks = df['Deaths'].values

In [ ]:
data_ns

In [ ]:
data_ks

In [ ]:
with pm.Model() as model4:
    alpha = pm.Gamma('alpha', alpha=4, beta=0.5)
    beta = pm.Gamma('beta', alpha=4, beta=0.5)
    xs = pm.Beta('xs', alpha, beta, shape=len(data_ns))
    ks = pm.Binomial('ks', n=data_ns, p=xs, observed=data_ks)
    idata4 = pm.sample(1000)

In [ ]:
pm.model_to_graphviz(model4)

In [ ]:
with model4:
    az.plot_trace(idata4)

In [ ]:
with model4:
    az.plot_posterior(idata4, var_names=['xs'])

In [ ]:
with model4:
    az.plot_posterior(idata4, var_names=['alpha', 'beta'])

In [ ]:
alphas = idata4.posterior['alpha'][0].values
betas = idata4.posterior['beta'][0].values

sample_posterior_x = pm.Beta.dist(alphas, betas)
sample_posterior_x = pm.draw(sample_posterior_x, 1000)
sample_posterior_x.mean()

In [ ]:
with model4:
    az.plot_posterior(sample_posterior_x)

In [ ]:
with pm.Model() as model5:
    alpha = pm.Exponential('alpha', lam=1)
    beta = pm.Exponential('beta', lam=1)
    mu = pm.Gamma('mu', alpha, beta)
    idata5 = pm.sample_prior_predictive(1000)

In [ ]:
sample_prior_mu = idata5.prior['mu']
sample_prior_mu.values.mean()

In [ ]:
def decorate_rate(ylabel='Density'):
    """Decorate the axes."""
    plt.xlabel('Goals per game (mu)')
    plt.ylabel(ylabel)
    plt.title('Distribution of goal scoring rate')
    legend()

In [ ]:
plot_kde(sample_prior_mu[0])
decorate_rate()

In [ ]:
df = pd.read_csv('Data/apache_sofa_all.csv')

In [ ]:
df['APACHE_II'].to_list()[:5]

In [ ]:
with pm.Model() as model6:
    alpha = pm.Exponential('alpha', lam=1)
    beta = pm.Exponential('beta', lam=1)
    mu = pm.Gamma('mu', alpha, beta)
    goals = pm.Poisson('goals', mu, observed=[16])

In [ ]:
pm.model_to_graphviz(model6)

In [ ]:
with model6:
    idata6 = pm.sample(1000)

In [ ]:
with model6:
    az.plot_trace(idata6)

In [ ]:
sample_post_mu = idata6.posterior['mu']
sample_post_mu.values.mean()

In [ ]:
plot_kde(sample_post_mu[0], label='mu posterior')
decorate_rate()

In [ ]:
df['Rozpoznania'].value_counts()

In [ ]:
df[df['Rozpoznania']=='CHF_niedokrwienna']['APACHE_II'].to_list()[:3]

In [ ]:
df[df['Rozpoznania']=='CHF_nieniedokrwienna']['APACHE_II'].to_list()[:3]

In [ ]:
with pm.Model() as model7:
    alpha = pm.Exponential('alpha', lam=1)
    beta = pm.Exponential('beta', lam=1)
    mu_A = pm.Gamma('mu_A', alpha, beta)
    mu_B = pm.Gamma('mu_B', alpha, beta)
    goals_A = pm.Poisson('goals_A', mu_A, observed=[9,25])
    goals_B = pm.Poisson('goals_B', mu_B, observed=[16,11])
    idata7 = pm.sample(1000)

In [ ]:
pm.model_to_graphviz(model7)

In [ ]:
with model7:
    az.plot_trace(idata7)

In [ ]:
mu_A = idata7.posterior['mu_A']
mu_B = idata7.posterior['mu_B']
mu_B.values.mean(), mu_A.values.mean()

In [ ]:
plot_kde(mu_A[0], label='mu_A posterior')
plot_kde(mu_B[0], label='mu_B posterior')
decorate_rate('Density')

In [ ]:
(mu_A > mu_B).values.mean()

In [ ]:
df['Rozpoznania'].value_counts()

In [ ]:
df[df['Rozpoznania']=='plyn_w_osierdziu ']

In [ ]:
data = dict(
            NDKW = df[df['Rozpoznania']=='CHF_niedokrwienna']['APACHE_II'].to_list()[:10],
            NNDKW = df[df['Rozpoznania']=='CHF_nieniedokrwienna']['APACHE_II'].to_list()[:10],
            PWO = df[df['Rozpoznania']=='plyn_w_osierdziu ']['APACHE_II'].to_list()[:2],
            NP = df[df['Rozpoznania']=='nadcisnienie_plucne']['APACHE_II'].to_list()[:2],
            SEP = df[df['Rozpoznania']=='sepsa']['APACHE_II'].to_list()[:1],
)

In [ ]:
with pm.Model() as model8:
    alpha = pm.Exponential('alpha', lam=1)
    beta = pm.Exponential('beta', lam=1)
    
    mu = dict()
    goals = dict()
    for name, observed in data.items():
        mu[name] = pm.Gamma('mu_'+name, alpha, beta)
        goals[name] = pm.Poisson(name, mu[name], observed=observed)
        
    idata8 = pm.sample(1000)

In [ ]:
viz = pm.model_to_graphviz(model8)
viz

In [ ]:
with model8:
    az.plot_trace(idata8, var_names=['alpha', 'beta'])

In [ ]:
sample_post_alpha = idata8.posterior['alpha']
sample_post_alpha.values.mean()

In [ ]:
sample_post_alpha = idata8.posterior['beta']
sample_post_alpha.values.mean()

In [ ]:
sample_post_mu_NDKW = idata8.posterior['mu_NDKW']
sample_post_mu_NNDKW = idata8.posterior['mu_NNDKW']
sample_post_mu_PWO = idata8.posterior['mu_PWO']
sample_post_mu_NP = idata8.posterior['mu_NP']
sample_post_mu_SEP = idata8.posterior['mu_SEP']
sample_post_mu_NDKW.values.mean(), sample_post_mu_NNDKW.values.mean(), sample_post_mu_PWO.values.mean(), sample_post_mu_NP.values.mean(), sample_post_mu_SEP.values.mean(), 

In [ ]:
plot_kde(sample_post_mu_NDKW[0], label='NDKW')
plot_kde(sample_post_mu_NNDKW[0], label='NNDKW')
plot_kde(sample_post_mu_PWO[0], label='PWO')
plot_kde(sample_post_mu_NP[0], label='NP')
plot_kde(sample_post_mu_SEP[0], label='SEP')
decorate_rate()

In [ ]:
(sample_post_mu_NDKW > sample_post_mu_PWO).values.mean()